In [41]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx

In [42]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/small2.wav"

In [43]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([-1.5259e-05,  4.5776e-05, -7.6294e-05,  ...,  0.0000e+00,
        -3.0518e-05,  4.5776e-05])


In [44]:
audio.shape

torch.Size([526073])

In [45]:
rfft_audio.apply_rfft(audio).shape

torch.Size([263037])

In [46]:
class RFFTModel(nn.Module):
    def __init__(self):
        super(RFFTModel,self).__init__()
    
    def forward(self, audio):
        rfft_magnitude = rfft_audio.apply_rfft(audio)
        return rfft_magnitude

In [47]:
model = RFFTModel()

In [48]:

export_option = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx_program = torch.onnx.dynamo_export(
    model,
    audio,
    # export_options = export_option
)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [49]:
onnx_program.save("rfft_model.onnx")

In [50]:
onnx_model = onnx.load("rfft_model.onnx")
onnx.checker.check_model(onnx_model)

In [51]:
import onnxruntime

In [52]:
ort_session = onnxruntime.InferenceSession("rfft_model.onnx")

In [53]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [54]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(audio)

In [55]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnx_output = ort_session.run(None,onnxruntime_input)

In [56]:
onnx_output[0]

array([2.106379  , 2.1144328 , 2.342691  , ..., 0.36589685, 0.90886784,
       0.5459296 ], dtype=float32)

In [57]:
rfft_direct = rfft_audio.apply_rfft(audio).numpy()

In [58]:
rfft_direct[:10]

array([2.1063848, 2.05767  , 2.3335254, 2.3117166, 2.1502516, 2.2714257,
       2.3398979, 2.3868306, 2.145785 , 1.9998296], dtype=float32)

In [59]:
onnx_output[0][:10]

array([2.106379 , 2.1144328, 2.342691 , 2.4306474, 2.1123135, 2.3142207,
       2.2664084, 2.4373214, 2.0836813, 1.3974711], dtype=float32)

In [60]:
import numpy as np

In [61]:
np.mean(np.abs(onnx_output[0] - rfft_direct))

1.2634953